<a href="https://colab.research.google.com/github/Illumaria/iti_funds_nav/blob/master/iti_ruse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [0]:
#url = 'https://www.londonstockexchange.com/exchange/prices-and-markets/ETFs/company-news.html?fourWayKey=LU1483649312LUUSDEUET'
url = 'https://www.londonstockexchange.com/exchange/prices-and-markets/rns/company-news.html?tidm=RUSE&isin=LU1483649312&newsType='

In [0]:
iti_resp = requests.get(url)

In [0]:
bs = BeautifulSoup(iti_resp.text)

In [191]:
dates = bs.find_all('span', class_="hour")
for date in dates:
  print(date.text.split()[0]+" "+date.text.split()[1]+" "+date.text.split()[2])

18 Feb 2020
17 Feb 2020
14 Feb 2020
13 Feb 2020
12 Feb 2020
11 Feb 2020
10 Feb 2020
07 Feb 2020
06 Feb 2020
05 Feb 2020
04 Feb 2020
03 Feb 2020
31 Jan 2020
30 Jan 2020
29 Jan 2020
28 Jan 2020
27 Jan 2020
24 Jan 2020
23 Jan 2020
22 Jan 2020


In [192]:
scripts = bs.find_all('a', title="View news announcement")
for script in scripts:
        print(script.get('href').split("'")[1])

/exchange/news/market-news/market-news-detail/RUSE/14427582.html
/exchange/news/market-news/market-news-detail/RUSE/14425887.html
/exchange/news/market-news/market-news-detail/RUSE/14424188.html
/exchange/news/market-news/market-news-detail/RUSE/14422348.html
/exchange/news/market-news/market-news-detail/RUSE/14420610.html
/exchange/news/market-news/market-news-detail/RUSE/14418887.html
/exchange/news/market-news/market-news-detail/RUSE/14417149.html
/exchange/news/market-news/market-news-detail/RUSE/14415333.html
/exchange/news/market-news/market-news-detail/RUSE/14413804.html
/exchange/news/market-news/market-news-detail/RUSE/14411813.html
/exchange/news/market-news/market-news-detail/RUSE/14409937.html
/exchange/news/market-news/market-news-detail/RUSE/14408107.html
/exchange/news/market-news/market-news-detail/RUSE/14406160.html
/exchange/news/market-news/market-news-detail/RUSE/14404367.html
/exchange/news/market-news/market-news-detail/RUSE/14402635.html
/exchange/news/market-new

In [193]:
print(len(scripts))
for i in range(len(scripts)):
  url2 = 'https://www.londonstockexchange.com' + scripts[i].get('href').split("'")[1]
  print('{}: {}'.format(i, url2))

20
0: https://www.londonstockexchange.com/exchange/news/market-news/market-news-detail/RUSE/14427582.html
1: https://www.londonstockexchange.com/exchange/news/market-news/market-news-detail/RUSE/14425887.html
2: https://www.londonstockexchange.com/exchange/news/market-news/market-news-detail/RUSE/14424188.html
3: https://www.londonstockexchange.com/exchange/news/market-news/market-news-detail/RUSE/14422348.html
4: https://www.londonstockexchange.com/exchange/news/market-news/market-news-detail/RUSE/14420610.html
5: https://www.londonstockexchange.com/exchange/news/market-news/market-news-detail/RUSE/14418887.html
6: https://www.londonstockexchange.com/exchange/news/market-news/market-news-detail/RUSE/14417149.html
7: https://www.londonstockexchange.com/exchange/news/market-news/market-news-detail/RUSE/14415333.html
8: https://www.londonstockexchange.com/exchange/news/market-news/market-news-detail/RUSE/14413804.html
9: https://www.londonstockexchange.com/exchange/news/market-news/marke

In [0]:
resp = requests.get(url2)
bs2 = BeautifulSoup(resp.text)

In [195]:
date = bs2.find('p', text = re.compile('DEALING DATE')).text
date.split()[2]

'21-Jan-2020'

In [196]:
nav = bs2.find('p', text = re.compile('NAV PER SHARE')).text
inav = float(nav.split()[4])
inav

32.3194

In [197]:
shares = bs2.find('p', text = re.compile('NUMBER OF SHARES')).text
n_shares = int(shares.split()[5])
n_shares

203000

In [198]:
total_nav = inav * n_shares
total_nav

6560838.2

In [201]:
nrows = len(scripts)
new_table = pd.DataFrame(columns=range(0,2), index = range(0, nrows))
#row_marker = 0
#for row in table.find_all('tr'):
#  column_marker = 0
#  columns = row.find_all('td')
#  for column in columns:
#    new_table.iat[row_marker,column_marker] = column.get_text()
#    column_marker += 1
#  row_marker+=1
df = new_table.assign(date = pd.to_datetime(new_table[0], dayfirst=True))
df.head(5)

,0,1,date
0,NaN,NaN,NaT
1,NaN,NaN,NaT
2,NaN,NaN,NaT
3,NaN,NaN,NaT
4,NaN,NaN,NaT


In [200]:
df = df.rename({1: "keyrate", 2: "date"}, axis='columns').drop(columns=[0]).dropna()
df['keyrate'] = pd.to_numeric(df['keyrate'])
df.index = pd.DatetimeIndex(df['date'])
type(df.index)

pandas.core.indexes.datetimes.DatetimeIndex